# Full Stack Quantum Computing
## Half and Full Adder Implementation

# Introduction

A quantum computer accepts input states that are coherent superpositions and subsequently manipulates these into corresponding superposition output states through a computation process. Quantum computation is a sequence of unitary transformations that form parallel data processing [1]. This allows quantum computers to efficiently solve problems which are intractable to classical computers. The quantum theory of computation reveals the fundamental connections between the laws of physics and the nature of mathematics [2,3].

Unitary operations are reversible therefore quantum arithmetic is built using reversible logical components [2]. Evidently, reversible quantum networks require additional storage for intermediate results. There is a delicate balance between number of computational steps and amount of auxilliary memory used in the implementation of quantum arithmetic.

It is often possible and even desirable to perform certain parts of quantum computations using classical conventional methods. This simplifies information storage and the information retrieval process. Thus leading to a practical distinction between quantum and classical variables and their appropriate usage and limitations [3].

This lab exercise attemps a demonstration of quantum arithmetic by constructing a quantum network (qu-net). The qu-net should demonstrate the optimal use of auxiliary memory by applying a computational reversal process to optimise space usage. The network is validated against the conventional method of arithmetic using truth tables. The space complexity of the network is discussed.

# BACKGROUND

A conventional or classical computer performs operations on classical bits using Boolean logic gates, while Quantum computers perform operations on qubits using quantum gates by means of unitary matrices. Classical logic gates are represented by truth tables with an arbitary number $n_i$ input and $n_o$ output width, quantum gates are represented by truth tables with an equal $n$ input and output width [4]. Classical half and full adders are illustrated by Fig. 1

In [ ]:
![image.png](attachment:image.png

# I. BASIC CONCEPTS

It may prove useful to define a string initialization of all quantum properties, where ZERO and ONE are string representations of $|0\rangle$ and $|1\rangle$ respectively. We may represent a Qbit with a two level quantum system as a class that inherits from the above mentioned QBaseState.

A quantum network in this context is the unitary evolution which takes an input state encoded in binary in the computational basis form into an output state [1,2]. Quantum arithmetic has several implementation methods. Elementary quantum gates are used as building components. The unitary evolution operation can be formally expressed as

$$ U_f |x\rangle \rightarrow |f(x)\rangle$$

where $ U_f$ is the unitary operator.

The addition of registers $|a\rangle$ and $|b\rangle)$ can be expressed as

$$|a,b,0\rangle \rightarrow|a,b,a+b\rangle$$

This expression demonstrates a naive approach using a anchilla bit ($|0\rangle$) to store the intermediate results [5]. A better approach is an operation, expressed as

$$|a,b,\rangle \rightarrow|a,a+b\rangle$$

which overwrites the result into one of the input registers [6]. The input $(a,b)$ can be obtained or reconstructed by reverse operating on the output without loss of information [3,6]. The rewritten register as well as the ouput register need to be sufficiently large to prevent overflows. A temporary register is also required to hold the carry of the operation to implement a full adder. The initialization is illustrated below.


In [ ]:
def registerInit(n):
        a = QuantumRegister(n) #first input number\n",
        b = QuantumRegister(n+1) #second input number and sum\n",
        c = QuantumRegister(n) # carry bits\n",
        cl = ClassicalRegister(n+1) # final output\n",
    
        qc = QuantumCircuit(a, b, c, cl)
    
        return qc, cl, a, b, c, n

# II. NETWORK ARCHITECTURE

The network achitecture is illustrated by Fig. 2.

![image.png](attachment:image.png)

The carries are calculated up until the most significant bit of the result, in a ripple formation. These operations are undone and subsequently the sum operations are performed. The reversal is denoted by the black bar on the left side.

# III. CARRY

The truth table for a carry gate is illustrated by below,

|Carry In| A  |B |Output|
|   :-:  | :-:|:-:|:-:|
|   0    |0|0|0     |
|   0    |0|1|0     |
|   0    |1|0|0     |
|   0    |1|1|1     |
|   1    |0|0|0     |
|   1    |0|1|1     |
|   1    |1|0|1     |
|   1    |1|1|1     |

The most significant bit of the result of $a+b$ is thus computed as follows: $$c_i  \leftarrow \text{$a_i$ AND $b_i$ AND $C_{i+1}$}$$

where $a_i$,& $b_i$ and $c_i$ are the $ith$ qubit of the registers above. The set of operations can be explained as
- if $a$ and $b$ are both in state $|1\rangle$ flip $c_{i+1}$
- if $a$ is in state $|1\rangle$, flip $b$
- finally, if both $c$ and $b$ are in state $|1\rangle$ flip output bit
    
Fig 3. illustrates the carry operation described above. 
   
![image.png](attachment:image.png)
 
Programmatically this is done as follows:

In [7]:
def carry(qc, a, b, c, n):
    
        for i in range(n-1):
                qc.ccx(a[i], b[i], c[i+1])
                qc.cx(a[i], b[i])
                qc.ccx(c[i], b[i], c[i+1])
                
    #   reverse operation 
        qc.ccx(a[n-1], b[n-1], b[n])
        qc.cx(a[n-1], b[n-1])
        qc.ccx(c[n-1], b[n-1], b[n])
        
        qc.cx(c[n-1], b[n-1])
       
        return qc, a, b, c, n

The last iteration of the carry gate is stored at $b[n]$ while the last carry bit resides at $c[n-1]$.
The operation performed on $b[n-1]$ is reversed and thus renders itself to be reused or available for later use.
All operation perfomed during the carry gate implementation are reversed to essentially reset the register to its initial state while simultaneously performing the sum operation."

# IV. SUM

The truth table for the sum gate is illustrated below,

|Carry In| A  |B |Output(B)|
|   :-:  | :-:|:-:|:-:|
|   0    |0|0|0     |
|   0    |0|1|1     |
|   0    |1|0|1     |
|   0    |1|1|0     |
|   1    |0|0|1     |
|   1    |0|1|0     |
|   1    |1|0|0     |
|   1    |1|1|1     |

The sum operation is achieved by applying a control gates whose operations are illustrated by Fig 4.

![image.png](attachment:image.png)

This represents the elementary sum operation where the gate takes in the carry and the inputs. This operation can be understood as follows

- If $carry$ is in state $|1\rangle$, flip $b$.
- If $a$ is in state $|1\rangle$, flip $b$ again.

Programmatically the combined operations are illustrated below."


In [10]:
def SUM(qc, a, b, c, n): #couldn't use camel case here (sum is a predefined function)\n",
   
    for i in range(n-1):
        qc.ccx(c[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
        qc.ccx(a[(n-2)-i], b[(n-2)-i], c[(n-1)-i])
    #   SUM Operation\n",
        qc.cx(c[(n-2)-i], b[(n-2)-i])
        qc.cx(a[(n-2)-i], b[(n-2)-i])
               
        return qc, a, b, c, n

# V. NETWORK COMPLEXITY AND SCALABILITY

The quantum network size depends on the input size and scales linearly. The IBM Qiskit simulator has $32$ qubits. Considering that the input is $2n$ bits, a carry register of $n$ bits and a classical register of $n+1$ bits to hold the results; a total of $4n + 1$ bits is required to implement the full adder (of which $3n$ are qubits). This means that $n$ cannot be greater than $7.75$, rounded down to nearest integer of $7$.

VI. FULL ADDER

For completeness the following documents the full adder implementation and the input initialisation process. An input of two binary numbers which are less than $8$ are initialised. The network size is initialised to the length of the longest input, this ensures that the registers can hold the final results results

In [12]:
def inputRequest():
        valid = False
        while not valid:
            first_number = input("Enter first binary number: ")
            second_number = input("Enter second binary number: ")
            
            if len(first_number) > 8 or len(second_number) > 8:
                print("Invalid input. Try again")
            else: valid = True
    
        if len(first_number) >  len(second_number):
            size = len(first_number)
        else:
            size = len(second_number)
    
        return first_number, second_number, size
    

The input is then transferred to the appropriate quantum registers. This is demonstrated programmatically below.

In [14]:
def quSet(first_number, second_number,a, b ,qc):
   
        count = 0
        for i in first_number:
                if i == "1": qc.x(a[len(first_number)- (count+1)])
                count+=1
   
        count = 0
        for i in first_number:
                if i == "1": qc.x(b[len(second_number) - (count+1)])
                count+=1
   
        return qc, a, b

The circuits output is obtained by measuring the result in register b and storing that in the classical register.

In [15]:
def measure(qc, b, cl, n):
        for i in range(n+1):
            qc.measure(b[i],cl[i])
        return qc

Finally the full adder implementation can be developed from the functions above.

In [18]:
def fullAdder():
   
    #     request inputs
        first_number, second_number, n = inputRequest()
    #     initalise registers using input size
        qc, cl, a, b, c, n = registerInit(n)
    #     transfer bit information into qubit registers
        qc, a, b = quSet(first_number, second_number,a, b ,qc)
    #     perform addition operation with carry
        qc, a, b, c, n = carry(qc, a, b, c, n)
        qc, a, b, c, n = SUM(qc, a, b, c, n)
    
        circuit = measure(qc, b, cl, n)
   
        num_shots = 1024
        job = execute(circuit, backend=Aer.get_backend('qasm_simulator'), shot = num_shots)
        
        qasm_result = job.result().get_counts()
        
        print(qasm_result)
       
        provider = IBMQ.get_provider('ibm-q')
    
        qcomp = provider.get_backend('ibmq_16_melbourne')
    
    
        job = execute(circuit, backend = qcomp)
    
        from qiskit.tools.monitor import job_monitor
    
        job_monitor(job)
        quam_result = job.result().get_counts()
        print("The measured Probabilities are:\n")
    
        print(quam_result)
    
        print("\nThe result with the maximum probability:\n")
    
        output = max(quam_result.items(), key=operator.itemgetter(1))[0]
        print(int(output,2))

        print("The transpiled circuit is: ")
        print(qc.qasm())
    

The complete circuit can be visualised below.

![image.png](attachment:image.png)

# VII. RESULT AND ANALYSIS

The truth table for a full adder is illustrated below,

|Carry In| A  |B  |Carry Out| Sum
|   :-:  | :-:|:-:|:-:      |:-:
|   0    |0   |0  |0        |0
|   0    |0   |1  |0        |1
|   0    |1   |0  |0        |1
|   0    |1   |1  |1        |0
|   1    |0   |0  |0        |1
|   1    |0   |1  |1        |0
|   1    |1   |0  |1        |0
|   1    |1   |1  |1        |1


This table is used to verify the results obtained from the simulations. The developed quantum network is simulated for a 3 qubit and 5 qubit circuit adding 1 bit numbers using the quantum hardware below.

![image.png](attachment:image.png)

The simulation is illustrated below:

Enter first binary number: 101
Enter second binary number: 101

In [ ]:
{'1010': 1024}
The transpiled circuit is: 
OPENQASM 2.0;
include \"qelib1.inc\";
qreg q65[3];
qreg q66[4];
qreg q67[3];
creg c7[4];
x q65[2];
x q65[0];
x q66[2];
x q66[0];
ccx q65[0],q66[0],q67[1];
cx q65[0],q66[0];
ccx q67[0],q66[0],q67[1];
ccx q65[1],q66[1],q67[2];
cx q65[1],q66[1];
ccx q67[1],q66[1],q67[2];
ccx q65[2],q66[2],q66[3];
cx q65[2],q66[2];
ccx q67[2],q66[2],q66[3];
cx q67[2],q66[2];
ccx q67[1],q66[1],q67[2];
cx q65[1],q66[1];
ccx q65[1],q66[1],q67[2];
cx q67[1],q66[1];
cx q65[1],q66[1];
ccx q67[0],q66[0],q67[1];
cx q65[0],q66[0];
ccx q65[0],q66[0],q67[1];
cx q67[0],q66[0];\n",
cx q65[0],q66[0];\n",
measure q66[0] -> c7[0];
measure q66[1] -> c7[1];
measure q66[2] -> c7[2];
measure q66[3] -> c7[3];

Enter numbers to simulate 3 qubit full adder

In [ ]:
fullAdder()

Below is the last result obtained

SAMPLE OUTPUT

In [ ]:
Enter first binary number: 0
Enter second binary number: 0

{'00': 1024}
    
Job Status: job has successfully run
   
{'00': 452, '01': 170, '10': 238, '11': 164}
    
Enter first binary number: 1
    
Enter second binary number: 1
    
{'10': 1024}
    
Job Status: job has successfully run
    
{'00': 210, '01': 209, '10': 361, '11': 237}

The results obtained above correlate with the truth table of the conventional full adder in particular the Qiskit Qasm simulator. The hardware simulated results present a distribution of possible states of the final output with the correct output of ${'00'}$ having the highest possibility. The other possibilities are due to errors in the hardware as evident from the hardware measurement errors depicted above. This forms as a validation of the implementation of the qu-net.

In [ ]:
Enter first binary number: 10101
Enter second binary number: 10101

In [ ]:
{'101010': 1024}
The transpiled circuit is: 
OPENQASM 2.0;
include "qelib1.inc";
qreg q72[5];
qreg q73[6];
qreg q74[5];
creg c8[6];
x q72[4];
x q72[2];
x q72[0];
x q73[4];
x q73[2];
x q73[0];
ccx q72[0],q73[0],q74[1];
cx q72[0],q73[0];
ccx q74[0],q73[0],q74[1];
ccx q72[1],q73[1],q74[2];
cx q72[1],q73[1];
ccx q74[1],q73[1],q74[2];
ccx q72[2],q73[2],q74[3];
cx q72[2],q73[2];
ccx q74[2],q73[2],q74[3];
ccx q72[3],q73[3],q74[4];
cx q72[3],q73[3];
ccx q74[3],q73[3],q74[4];
ccx q72[4],q73[4],q73[5];
cx q72[4],q73[4];
ccx q74[4],q73[4],q73[5];
cx q74[4],q73[4];
ccx q74[3],q73[3],q74[4];
cx q72[3],q73[3];
ccx q72[3],q73[3],q74[4];
cx q74[3],q73[3];
cx q72[3],q73[3];
ccx q74[2],q73[2],q74[3];
cx q72[2],q73[2];
ccx q72[2],q73[2],q74[3];
cx q74[2],q73[2];
cx q72[2],q73[2];
ccx q74[1],q73[1],q74[2];
cx q72[1],q73[1];
ccx q72[1],q73[1],q74[2];
cx q74[1],q73[1];
cx q72[1],q73[1];
ccx q74[0],q73[0],q74[1];
cx q72[0],q73[0];
ccx q72[0],q73[0],q74[1];
cx q74[0],q73[0];
cx q72[0],q73[0];
measure q73[0] -> c8[0];
measure q73[1] -> c8[1];
measure q73[2] -> c8[2];
measure q73[3] -> c8[3];
measure q73[4] -> c8[4];
measure q73[5] -> c8[5];

In [ ]:
# Enter numbers to simulate 5 qubit full adder

In [ ]:
fullAdder()

Below is the last result obtained

"An rather interesting development is observed when the circuit is reversed, when each gate is applied in the reversed order. Two kinds of outputs are observed depending on the input. When $a \geqslant b$, the output produced is $(a, a-b)$ while the output $(a, 2^{n+1}+(a-b))$ is observed when $a<b$. It is observed that the most significant qubit in the second register will always be in the state $|1\rangle$ when $a<b$. This allows for the comparison of the inputs, it can thus be deduced with certainty whether $a$ is bigger than $b$ or not by simply inspecting the overflow. This concept provides additional information about the inputs and can be used in the effecient implementation of complex digital communication. Conventional arithmetic lacks the ability to predict the input state by observing the output state.

# VIII. CONCLUSION

A quantum network for general and scalable quantum addition is implemented and analysed. The qu-net is simulated using IBM Qiskit simulator, and a quantum hardware device. The simulation results obtained from the IBM Qiskit simulator and hardware are validated against the classical method. The results from the quantum device are analysed, some errors are observed as expected. The implementation requires $4n+1$ bits of which $3n$ are qubits.The size of the input is limited to $7$ qubits and $3$ qubits so as to not exceed the $32$ qubit limitation on IBM Qiskit simulator and the $15$ qubit quantum hardware respectively. The qu-net complexity and scalability is discussed and a modest linear scaling is achieved.

# IX. ACKNOWLEDGMENTS

[1] D. Deutsch, Proc. R. Soc. London A 400, 97 (1985).
   
[2] D. Deutsch and R. Jozsa, Proc. R. Soc. London A 439, 553 "(1992); E. Bernstein and U. Vazirani (unpublished); D. S. Simon, in Proceedings of the 35th Annual Symposium on the Foundations of Computer Science, edited by S. Goldwasser (IEEE Computer Society Press, Los Alamitos, CA, 1994).
    
    
"[3] P. W. Shor, in Proceedings of the 35th Annual Symposium on the Theory of Computer Science.
    
[4] A. Barenco, C.H. Bennett, R. Cleve, D. P. DiVicenzo, N. Margolus, P. Shor, T. Sleator, J. Smolin, and H. Weinfurter, Phys. Rev. A 52, 3457 ~1995.
    
[5] R. Wille and R. Drechsler, Towards a Design Flow for Reversible Logic, Springer, Dordrecht, 2010. DOI: 10.1007/978-90-481-9579-4. 10, 56
    
[6] R. Jozsa, Quantum algorithms, In: D. Bouwmeester, A. Ekert, and A. Zeilinger, The Physics of Quantum Information, Springer Verlag, Berlin, pp. 104–126, 2000. 62